In [60]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [8]:
data = pd.read_csv('data/mnist_train_small.csv', header=None)

In [10]:
data = data.reindex(np.random.permutation(data.index))

In [29]:
data[np.arange(1,785)] = data[np.arange(1,785)]/255

In [31]:
tr_x = data[np.arange(1,785)].head(18000)
tr_y = data[0].head(18000)

te_x = data[np.arange(1,785)].tail(2000)
te_y = data[0].tail(2000)

In [43]:
def my_input_fn(features, targets, batch_size=100, num_epochs=10):
    features = {'pixels': features.values}
    targets = np.array(targets)
    
    ds = tf.data.Dataset.from_tensor_slices((features, targets)).batch(batch_size).repeat(num_epochs)
    return ds.make_one_shot_iterator().get_next()

In [42]:
with tf.Session() as sess:
    _ = sess.run(my_input_fn(tr_x, tr_y))

In [49]:
my_opt = tf.train.AdagradOptimizer(learning_rate=.01)
my_opt = tf.contrib.estimator.clip_gradients_by_norm(my_opt, 5.0)
lc = tf.estimator.LinearClassifier(optimizer=my_opt,
                                   n_classes=10,
                                   feature_columns=set([tf.feature_column.numeric_column('pixels', shape=784)]))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpd10alh4x', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f77528f5c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [66]:
lc.train(steps=30000, input_fn=lambda: my_input_fn(tr_x, tr_y))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpd10alh4x/model.ckpt-311
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 312 into /tmp/tmpd10alh4x/model.ckpt.
INFO:tensorflow:loss = 42.064857, step = 311
INFO:tensorflow:global_step/sec: 358.816
INFO:tensorflow:loss = 30.263601, step = 411 (0.280 sec)
INFO:tensorflow:global_step/sec: 434.008
INFO:tensorflow:loss = 38.219357, step = 511 (0.230 sec)
INFO:tensorflow:global_step/sec: 440.588
INFO:tensorflow:loss = 26.75999, step = 611 (0.227 sec)
INFO:tensorflow:global_step/sec: 434.03
INFO:tensorflow:loss = 44.659874, step = 711 (0.230 sec)
INFO:tensorflow:global_step/sec: 438.734
INFO:tensorflow:loss = 32.856407, step = 811 (0.229 sec)
INFO:tensorflow:global_step/sec: 438.038
INFO:tensorflow:loss = 44.907543, step

In [67]:
prd = lc.predict(input_fn=lambda: my_input_fn(te_x, te_y, 2000, 1))

prd = np.array([pred['class_ids'][0] for pred in prd])
accuracy_score(te_y, prd)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpd10alh4x/model.ckpt-2111
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.902